### Experiment : Set up optuna with neptune for Hyper parameter and architecture buildinng
### Objectives
https://www.kaggle.com/mistag/keras-model-tuning-with-optuna
1. batch size =[8,16,32]
2. learining rate = [0.1,0.01,0.0001,0.00001]
3. units = [8,16,32,64,128]
4. number of layers in architecture [4,5,6]
5. optimization = [adam,]
6. epochs = [10,20]

### Observations

1. 
2. 
3. 

### Solutions and further investigation

1. 
2. 
3. 

Categorical parameters – uses the trials.suggest_categorical() method. You need to provide the name of the parameter and its choices.

Integer parameters – uses the trials.suggest_int() method. You need to provide the name of the parameter, low and high value.

Float parameters – uses the trials.suggest_float() method. You need to provide the name of the parameter, low and high value.

Continuous parameters – uses the trials.suggest_uniform() method. You need to provide the name of the parameter, low and high value.

Discrete parameters – uses the trials.suggest_discrete_uniform() method. You need to provide the name of the parameter, low value, high value, and step of discretization

    #### samplers
    GridSampler – It uses a grid search. The trials suggest all combinations of parameters in the given search space during the study.

    RandomSampler – It uses random sampling. This sampler is based on independent sampling.

    TPESampler – It uses the TPE (Tree-structured Parzen Estimator) algorithm.
    

### Load Modeules and datasets

In [6]:
import pandas as pd
import numpy as np
import random as rn
np.random.seed(0)
rn.seed(0)
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV,train_test_split
# from keras import layers
from sklearn import metrics
# from keras import regularizers
# from keras import models,optimizers
# from keras.models import Sequential
# from tensorflow.python.keras.optimizer_v2.adam import Adam
# from keras.layers import Dense,Input,SpatialDropout1D,Dropout,Flatten, SimpleRNN,LSTM,RNN,GRU
# from keras.layers.embeddings import Embedding
# from keras.preprocessing import sequence
# from keras.preprocessing.text import Tokenizer
# from keras.utils.vis_utils import plot_model
# from keras.callbacks import EarlyStopping

import re
import gensim
pd.set_option('display.max_colwidth', 1000)
# import helper function script
import sys
sys.path.insert(1,'G:\\Github\\Sinhala-Hate-Speech-Detection')
import utills
import sinhala_stemmer
import TextClassifier

In [7]:
import neptune
#import neptune.new as neptune
from neptunecontrib.monitoring.metrics import expand_prediction, log_class_metrics, log_binary_classification_metrics, log_classification_report,log_confusion_matrix,log_prediction_distribution
from neptunecontrib.api import log_table
from neptunecontrib.monitoring.keras import NeptuneMonitor
import neptunecontrib.monitoring.optuna as opt_utils 
import os
from dotenv import load_dotenv

load_dotenv()
NEPTUNE_PROJECT= os.getenv('NEPTUNE_PROJECT')
NEPTUNE_API_TOKEN = os.getenv(('NEPTUNE_API_TOKEN'))
neptune.init(project_qualified_name= NEPTUNE_PROJECT,api_token=NEPTUNE_API_TOKEN) 

# import neptune.new.integrations.optuna as optuna_utils

# neptune_callback = optuna_utils.NeptuneCallback(run)

Project(codekcg23/Research-Experiments)

In [8]:
TC = TextClassifier.TextClassifier()

Load data

X train (4526,) Y train (4526,) X test (1940,) Y test (1940,)
dictionary size:  17577


In [ ]:
def create_model(trial, TC,model_type=None):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.optimizer_v2.adam import Adam
    from tensorflow.keras.layers import Dense,Input,SpatialDropout1D,Dropout,Flatten, SimpleRNN,LSTM,RNN,GRU,Bidirectional
    from tensorflow.keras.layers.embeddings import Embedding
    from tensorflow.keras.layers import LeakyReLU

    leaky_relu = LeakyReLU(alpha=0.01)

    layer = trial.suggest_categorical("layers",[0,1,2,3])
    unit = trial.suggest_categorical("units",[8,16,32,64,128])
    dropout_rate = trial.suggest_float("dropout_rates",[0.0, 0.4, 0.1])
    recurr_dropout = trial.suggest_float("recurrent_dropout_rates",[0.0, 0.3, 0.1])
    activ = trial.suggest_categorical("activation", ["relu",  leaky_relu, "elu","tanh"])

    print("Build ML model")
    model = Sequential()
    if(TC.EMBEDDING == None):
        model.add(Embedding(output_dim=TC.EMBEDDING_SIZE, 
                        input_dim=TC.LEN_VOCAB, 
                        input_length=TC.MAX_SEQ_LEN,
                       # weights=[TC.emb_matrix], # Additionally we give the Wi
                        trainable=TC.trainable))

    else:
        model.add(Embedding(output_dim=TC.EMBEDDING_SIZE, 
                        input_dim=TC.LEN_VOCAB, 
                        input_length=TC.MAX_SEQ_LEN,
                        weights=[TC.emb_matrix], # Additionally we give the Wi
                        trainable=TC.trainable)) # Don't train the embeddings - just use GloVe embeddings
    # We can start with pre-trained embeddings and then fine-tune them using our data by setting trainable to True
    if(model_type=="RNN"):
        if layer!=0:
            for i in range(layer):
                model.add(SimpleRNN(unit, activation=activ,dropout=dropout_rate, recurrent_dropout=recurr_dropout,return_sequences=True))
        model.add(SimpleRNN(unit, activation=activ,dropout=dropout_rate, recurrent_dropout=recurr_dropout))

    elif(model_type == "LSTM"):
        if layer!=0:
            for i in range(layer):
                model.add(LSTM(unit, activation=activ,dropout_rate, recurrent_dropout=recurr_dropout,return_sequences=True))
        model.add(LSTM(unit, activation=activ,dropout_rate, recurrent_dropout=recurr_dropout))
    elif(model_type == "GRU"):
        if layer!=0:
            for i in range(layer):
                model.add(GRU(unit, activation=activ,dropout_rate, recurrent_dropout=recurr_dropout,return_sequences=True))
        model.add(GRU(unit, activation=activ,dropout_rate, recurrent_dropout=recurr_dropout))
    elif(model_type == "BiLSTM"):
         if layer!=0:
            for i in range(layer):
                model.add(Bidirectional(LSTM(unit, activation=activ,dropout=dropout_rate, recurrent_dropout=recurr_dropout,return_sequences=True)))
        model.add(Bidirectional(LSTM(unit, activation=activ,dropout=dropout_rate, recurrent_dropout=recurr_dropout)))
    elif(model_type == "BiGRU"):
        if layer!=0:
            for i in range(layer):
                model.add(Bidirectional(GRU(unit, activation=activ,dropout=dropout_rate, recurrent_dropout=,return_sequences=True)))
        model.add(Bidirectional(GRU(unit, activation=activ,dropout=dropout_rate, recurrent_dropout=recurr_dropout)))
    #model.add(Dropout(0.2))
    model.add(Dense(unit, activation=activ))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation='sigmoid'))
    optimizer_adam = Adam(learning_rate=TC.lr)
    model.compile(loss='binary_crossentropy',
                optimizer=optimizer_adam,
                metrics=['acc'])
    print(model.summary())
    TC.model = model
    return model

In [ ]:
TC_W.EPOCHS =3
TC_W.BATCH_SIZE =16
TC_W.lr = 0.001
TC_W.tag = "test optuna"
MODEL_TYPE = "dense"
rnn_model = get_model("RNN")
rnn_model, hist = train_model(rnn_model)

In [ ]:
Y_pred = TC_W.model_evaluate(rnn_model,hist)
TC_W.save_predictions(Y_pred,TC_W.tag)

In [ ]:
import optuna
from optuna import trial
import optuna.integration.TFKerasPruningCallback
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
def objective(trial):  # trial is object
    # from tensorflow.keras.callbacks import LearningRateScheduler
    # from tensorflow.keras.optimizers.schedules import ExponentialDecay
 
    model = create_model(trial,TC_W,MODEL_TYPE)
    # scheduler = ExponentialDecay(1e-3, 400*((len(train)*0.8)/BATCH_SIZE), 1e-5)
    # lr = LearningRateScheduler(scheduler, verbose=0)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, verbose=1,mode='min',restore_best_weights=True)
    pruning = TFKerasPruningCallback(trial, "val_loss")
    lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-5,mode='min', verbose=verbose)
    callbacks_list =[early_stopping,lr,pruning]
        # Fit the model on the training data.
        # The TFKerasPruningCallback checks for pruning condition every epoch.
    model.fit(
        X_train,
        y_train,
        batch_size=BATCH_SIZE,
        callbacks=callbacks_list,
        epochs=TC_W.EPOCHS,
        validation_data=0.1,
        verbose=1,
    )
    #score = model.evaluate(TC_W.X_test, TC_W.Y_test, verbose=0)
    #return score[1]
    validation_loss = np.min(h.history['val_loss'])
                 
    return validation_loss
    # f1_score = model_performence(model)
    # return f1_score


In [ ]:
N_TRIALS = 3
neptune_callback = opt_utils.NeptuneMonitor(
    # plots_update_freq=10,  # create/log plots every 10 trials
    # log_plot_slice=False,  # do not create/log plot_slice
    # log_plot_contour=False,  # do not create/log plot_contour
)

study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective,n_trials=N_TRIALS,callbacks=[neptune_callback])  # number of trials
 
# Log Optuna charts and study object after the sweep is complete
# optuna_utils.log_study_metadata(study,
#                                 run,  
#                                 log_plot_contour=False)

### other models

In [ ]:
# model creation
def create_lstm_model(trial):

    x0 = tf.keras.layers.Input(shape=(train.shape[-2], train.shape[-1]))  

    lstm_layers = 4
    lstm_units = np.zeros(lstm_layers, dtype=np.int)
    lstm_units[0] = trial.suggest_int("lstm_units_L1", 768, 1536)
    lstm = Bidirectional(keras.layers.LSTM(lstm_units[0], return_sequences=True))(x0)
    for i in range(lstm_layers-1):
        lstm_units[i+1] = trial.suggest_int("lstm_units_L{}".format(i+2), lstm_units[i]//2, lstm_units[i])
        lstm = Bidirectional(keras.layers.LSTM(lstm_units[i+1], return_sequences=True))(lstm)    
    dropout_rate = trial.suggest_float("lstm_dropout", 0.0, 0.3)
    lstm = Dropout(dropout_rate)(lstm)
    dense_units = lstm_units[-1]
    # try different activations
    activation = trial.suggest_categorical("activation", ["relu", "selu", "elu", "swish"])
    lstm = Dense(dense_units, activation=activation)(lstm)
    lstm = Dense(1)(lstm)

    model = keras.Model(inputs=x0, outputs=lstm)
    metrics = ["mae"]
    model.compile(optimizer="adam", loss="mae", metrics=metrics)
    
    return model

In [ ]:
def train_model(model,train_seq,test_seq,Y_train,Y_test,tag):
  
  #define callbacks
  
  early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, verbose=1,mode='min',restore_best_weights=True)
  #callbacks_list = [early_stopping]

  #model training
  print("started training")
  hist = model.fit(train_seq, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping],validation_split=0.1, shuffle=False, verbose=2)
  train_loss, train_acc = model.evaluate(train_seq, Y_train,batch_size=BATCH_SIZE, verbose=1)
  print("train loss - ",train_loss," train acc- ",train_acc)
  test_loss, test_acc = model.evaluate(test_seq,Y_test,batch_size=BATCH_SIZE, verbose=1)
  print("test loss - ",test_loss," test acc- ",test_acc)

  # plot loss during training
  #from matplotlib import pyplot
  print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
  plt.subplot(211)
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.ylabel("Cross Entropy Loss")
  plt.plot(hist.history['loss'],lw=2.0, color='b', label='train')
  plt.plot(hist.history['val_loss'],lw=2.0, color='r', label='validation')
  plt.legend(loc='upper right')
  # plot accuracy during training
  plt.subplot(212)
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.plot(hist.history['accuracy'],lw=2.0, color='b', label='train')
  plt.plot(hist.history['val_accuracy'],lw=2.0, color='r', label='validation')
  plt.legend(loc='upper right')
  plt.show()
  #(model.predict(x) > 0.5).astype("int32")
  #np.where(y_pred > threshold, 1,0)
  Y_pred =(model.predict(test_seq) > 0.5).astype("int32")# model.predict_class(test_seq)
  Y_pred = np.squeeze(Y_pred)
  log_result(Y_test,Y_pred,"fasttext","RNN",tag)
  print(classification_report(Y_test,Y_pred))
  #Y_pred = np.squeeze(Y_pred)
  utills.confusion_Matrix(Y_test,Y_pred)
  utills.PlotRocAuc(Y_test,Y_pred,'blue',"RNN")
  return model,Y_pred

In [ ]:
def log_result(Y_test,Y_pred,feature_name,model_name,tag):
    print("========= Eperiment - ",tag," =========")
    neptune.create_experiment(tag,params=PARAMS)
    neptune.append_tag(['DL experiment',feature_name,model_name,tag])
    
    log_class_metrics(Y_test, Y_pred)
    log_confusion_matrix(Y_test, Y_pred)
    log_classification_report(Y_test, Y_pred)

### Optuna

Keras: optuna.integration.KerasPruningCallback
TensorFlow optuna.integration.TensorFlowPruningHook
tf.keras optuna.integration.TFKerasPruningCallback

In [ ]:
study = optuna.create_study(pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=2, reduction_factor=4, min_early_stopping_rate=1))
study.optimize(objective_with_prune, timeout=timeout)

In [ ]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=100)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

In [ ]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_optimization_history(study)

In [ ]:
trials_df = study.trials_dataframe()
trials_df

In [ ]:
joblib.dump(study, 'optuna_searches/study.pkl'

In [ ]:
#  lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
# #         lr = WarmupExponentialDecay(lr_base=1e-3, decay=1e-5, warmup_epochs=30)
#         es = EarlyStopping(monitor="val_loss", patience=60, verbose=1, mode="min", restore_best_weights=True)
    
#         checkpoint_filepath = f"folds{fold}.hdf5"
#         sv = keras.callbacks.ModelCheckpoint(
#             checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
#             save_weights_only=False, mode='auto', save_freq='epoch',
#             options=None
#         )

#         model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, es, sv])
#         #model.save(f'Fold{fold+1} RNN Weights')
#         test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

In [ ]:
# Log single String
run['aux/text'] = 'text I keep track of, like query or tokenized word'

# Log series of String to one log
for epoch in range(epochs_nr):
    token = str(...)
    run['train/tokens'].log(token)
## CSV file
#run['test/preds'].upload('path/to/test_preds.csv')
from neptune.new.types import File

# Pandas DataFrame 
#run['data/sample'].upload(File.as_html(sample_df))